In [30]:
import numpy as np
import copy
from pprint import pprint
import math
import sys
np.set_printoptions(threshold=np.inf)

sys.setrecursionlimit(10000)

In [96]:
sudoku = np.zeros((4,4))
HansonMatrix = np.zeros((729,324))
sudoku = np.array([[1,2,3,4],[3,4,1,2],[4,1,2,3],[2,3,4,1]])
grid = np.array([[0,8,0,0,0,4,0,5,0],
                [0,6,0,2,0,0,0,0,0],
                [5,0,2,0,7,0,1,0,0],
                [0,0,6,0,0,0,0,0,0],
                [2,0,1,9,0,0,0,4,0],
                [0,0,0,0,8,0,0,0,9],
                [0,0,0,0,0,3,7,0,0],
                [4,0,9,8,0,0,0,1,0],
                [0,5,0,0,0,0,0,0,0]])
#pprint(HansonMatrix)

In [5]:

class Node:
    def __init__(self,row = None, column = None, count = None):
        self.left = self
        self.right = self
        self.up = self
        self.down = self
        self.colheader = self
        self.rowheader = self
        self.row = row
        self.column = column
        self.nodeCount = count
        self.visit = 0

In [39]:
def rowNodes(node):
    list = []
    list.append(node)
    a = node.column
    current = node.right
    while (current.column != a):
        list.append(current)
        current = current.right
    return list

def columnNodes(node):
    list = []
    a = node.row
    #print(a)
    current = node.up
    #print(current.row)
    while (current.row != a):
        if (current.row == -1):
            current = current.up
            continue
        list.append(current.row)
        current = current.up
    
    return list

def initialize(HansonMatrix,numbers):

    for column in range(numbers*numbers):
        for row in range(column*numbers,column*numbers + numbers):
            HansonMatrix[row][column] = 1
    
    for i in range(numbers):
        for chunk in range(i*numbers, (i+1)*numbers):
            for j in range(numbers):
                row = chunk*numbers + j
                column = i*numbers + j + numbers*numbers
                HansonMatrix[row][column] = 1

    for i in range(numbers):
        for j in range(numbers*numbers):
            row = i*numbers*numbers + j
            column = j + 2*numbers*numbers
            HansonMatrix[row][column] = 1

    for i in range(np.sqrt(numbers).astype(int)):
        for j in range(np.sqrt(numbers).astype(int)):
            for k in range(np.sqrt(numbers).astype(int)):
                for w in range(np.sqrt(numbers).astype(int)):
                    for r in range(numbers):
                        column = 3*numbers*numbers + i*numbers*np.sqrt(numbers).astype(int) + k*numbers + r
                        row = i*numbers*numbers*np.sqrt(numbers) + j*numbers*numbers + k*numbers*np.sqrt(numbers).astype(int) + w*numbers + r
                        #print(f"{row}" + " " +  f"{column}")
                        row = row.astype(int)
                        column = column.astype(int)
                        #print(row)
                        HansonMatrix[row][column] = 1

def initial_links(HansonMatrix,HeaderList,ColumnList):

    rows = len(HansonMatrix)
    columns = len(HansonMatrix[0])
    for i in range(rows):       
        for j in range(columns):
            if HansonMatrix[i][j]:
                HeaderList[j].nodeCount += 1
                ColumnList[i].nodeCount += 1

                newNode = Node(row=i,column=j)

                newNode.colheader = HeaderList[j]
                newNode.rowheader = ColumnList[i]

                newNode.up = HeaderList[j].up
                HeaderList[j].up.down = newNode
                HeaderList[j].up = newNode
                newNode.down = HeaderList[j]
                prevNode = newNode

                newNode.left = ColumnList[i].left
                ColumnList[i].left.right = newNode
                ColumnList[i].left = newNode
                newNode.right = ColumnList[i]
            
def fitting_sudoku(HansonMatrix,grid,RowList,numbers):
    cleanup = []
    rows = len(grid)
    columns = len(grid[0])
    for i in range(rows):
        for j in range(columns):
            value = grid[i][j]
            if value == 0:
                continue
            row_indice = numbers*numbers*i + numbers*j + value - 1
            current = RowList[row_indice]
            nodes = rowNodes(current)
            for node in nodes:
                #print(node.row)
                cleanup += columnNodes(node)
                #print(columnNodes(node))
    
    clear = list(set(cleanup))
    #print(clear)
    for i in clear:
        HansonMatrix[i,:] = 0

def sudoku_links(HansonMatrix,grid,RowList,HeaderList):
    initial_links(HansonMatrix,HeaderList,RowList)
    fitting_sudoku(HansonMatrix,grid,RowList)

def printing(HeaderList):
    for node in HeaderList:
        current = node.down
        while (current != node):
            print(current.row)
            print(current.column)

def cover(Node):
    header = Node.header
    header.left.right = header.right
    header.right.left = header.left

    current = Node.down

    while current != header:
        right = current.right
        while right != current:
            right.up.down = right.down
            right.down.up = right.up
            right.header.nodeCount -= 1
            right = right.right
        current = current.down

def uncover(Node):
    header = Node.header

    up = header.up
    while up != header:
        left = up.left
        while left != up:
            left.up.down = left
            left.down.up = left
            left.header.nodeCount += 1
            left = left.left
        up = up.up

    header.left.right = header
    header.right.left = header

def minColumn(centralNode):
    current = centralNode.right
    a = current.nodeCount
    node = current
    while current != centralNode:
        if current.nodeCount < a:
            a = current.nodeCount
            node = current
        current = current.right
    return node

def print_sol(solutions):
    for node in solutions:
        print(f"{node.row} +  {node.column}")
        #print(node.column)

def solve(k,HeaderList,solutions,centralNode,RowList):
    
    if centralNode.right == centralNode:
        print_sol(solutions)
        return
    
    column = minColumn(HeaderList)
    cover(column)

    # print("Checking first cover")
    # print_all_nodes(centralNode)
    # print_rows(RowList)

    row = column.down

    while row != column:

        solutions.append(row)

        right = row.right
        while right != row:
            print(f"{right.row} + {right.column}")
            print("HEREEEE")
            cover(right)
            right = right.right

        solve(k+1,HeaderList,solutions,centralNode,RowList)
        solutions.pop()
        column = row.column

        left = row.left
        while left != row:
            ("HERASDFAD")
            uncover(left)
            left = left.left

        row = row.down
    
    uncover(column)

def print_all_nodes(node):
    if not node:
        return
    
    print("Printing all nodes in the structure column wise:")
    start_node = node
    
    # Start by traversing the columns
    col_node = start_node
    while True:
        print(f" Node of Interest -    Row: {col_node.row}, Column: {col_node.column}")
        print ("now going downwards and checking this column")
        
        # Now traverse the rows for this column
        row_node = col_node.down
        while row_node != col_node:
            print(f"    Row: {row_node.row}, Column: {row_node.column}")
            row_node = row_node.down
        
        col_node = col_node.right
        if col_node == start_node:
            break

def print_rows(RowList):
    for node in RowList:
        print("Printing all nodes in the structure row wise:")
        start_node = node
    
        # Start by traversing the rows
        row_node = start_node
        while True:
            print(f" Node of Interest -    Row: {row_node.row}, Column: {row_node.column}")
            print ("now going sideways and checking this row")
            
            # Now traverse the rows for this column
            col_node = row_node.right
            while col_node != row_node:
                print(f"    Row: {col_node.row}, Column: {col_node.column}")
                col_node = col_node.right
            
            if row_node == col_node:
                break

def print_nodeCount(HeaderList):
    vector = []
    for node in HeaderList:
        vector.append(node.nodeCount)
    print(vector)

def print_all_links(Node):
    print("*****************")
    print(f"Node rowheader {Node.row} and {Node.column} is {Node.rowheader.row} , {Node.rowheader.column}")
    print(f"Node colheader  {Node.row} and {Node.column} is {Node.colheader.row} , {Node.colheader.column}")
    print(f"Node count for {Node.row} and {Node.column} is {Node.nodeCount}")
    print(f"Node above {Node.row} and {Node.column} is {Node.up.row} , {Node.up.column}")
    print(f"Node below {Node.row} and {Node.column} is {Node.down.row} , {Node.down.column}")
    print(f"Node right {Node.row} and {Node.column} is {Node.right.row} , {Node.right.column}")
    print(f"Node left {Node.row} and {Node.column} is {Node.left.row} , {Node.left.column}")

def printt_matrix(Node):
    if (Node.visit):
        return
    print_all_links(Node)
    Node.visit = 1
    printt_matrix(Node.right)
    printt_matrix(Node.down)
    printt_matrix(Node.left)
    printt_matrix(Node.up)

def reset(Node):
    if (Node.visit == 0):
        return
    Node.visit = 0
    reset(Node.right)
    reset(Node.left)
    reset(Node.up)
    reset(Node.down)

def print_matrix(Node):
    printt_matrix(Node)
    reset(Node)

def print_func_1(Node,testMatrix):
    if Node.row < 0 or Node.column < 0:
        return
    testMatrix[Node.row][Node.column] = 1

def print_func_2(Node,testMatrix):
    if (Node.visit):
        return
    print_func_1(Node,testMatrix)
    Node.visit = 1
    print_func_2(Node.right,testMatrix)
    print_func_2(Node.down,testMatrix)
    print_func_2(Node.left,testMatrix)
    print_func_2(Node.up,testMatrix)

def reset(Node):
    if (Node.visit == 0):
        return
    Node.visit = 0
    reset(Node.right)
    reset(Node.left)
    reset(Node.up)
    reset(Node.down)

def printHanson(testMatrix,centralNode):
    print_func_2(centralNode,testMatrix)
    reset(centralNode)

In [92]:
def remove(Node):
    Node.left.right = Node.right
    Node.right.left = Node.left
    Node.up.down = Node.down
    Node.down.up = Node.up
    if Node.row >= 0 and Node.column >=0:
        Node.rowheader.nodeCount -= 1
        Node.colheader.nodeCount -= 1

def fix_column_node(Node):
    current = Node.up
    while (current!=Node):
        remove(current)
        current = current.up
    remove(Node)

def fix_row(ColumnList,i,Solutions):
    if (ColumnList[i].nodeCount == 4):
        Solutions.append(ColumnList[i].row)

        header = ColumnList[i]
        # header.up.down = header.down
        # header.down.up = header.up
        current = header.right
        while current != header:
            fix_column_node(current)
            current = current.right
        remove(ColumnList[i])
    else:
        print("Ran into an exception")

def solve_as_much_as_possible(Solutions,centralNode,ColumnList):

    RowCleanUp = []
    iterations = 0
    curr = len(Solutions)
    while(iterations < 2):
    
        print(len(Solutions))
        current = centralNode.down
        while current != centralNode:
            if (current.nodeCount < 4):
                RowCleanUp.append(current)
            current = current.down

        for node in RowCleanUp:
            current = node.right
            while current != node:
                remove(current)
                current = current.right
            remove(node)

        current = centralNode.right
        while current != centralNode:
            if (current.nodeCount == 1):
                fix_row(ColumnList,current.down.row,Solutions)
            current = current.right

        if curr == len(Solutions):
            iterations+= 1
        else:
            iterations = 0
            curr = len(Solutions)

def traverse(ogNode, old_to_new):
    if not ogNode or ogNode.visit == 1:
        return old_to_new.get(ogNode)

    # Create a deep copy of the current node and store it in the map
    #CopyNode = Node.__class__(Node.value)  # Use the class of the Node to create a new instance
    CopyNode = Node(row = ogNode.row, column = ogNode.column, count = ogNode.nodeCount)
    old_to_new[ogNode] = CopyNode

    # Mark the current node as visited
    ogNode.visit = 1

    # Recursively copy all connected nodes
    CopyNode.colheader = traverse(ogNode.colheader, old_to_new)
    CopyNode.rowheader = traverse(ogNode.rowheader, old_to_new)
    CopyNode.up = traverse(ogNode.up, old_to_new)
    CopyNode.down = traverse(ogNode.down, old_to_new)
    CopyNode.right = traverse(ogNode.right, old_to_new)
    CopyNode.left = traverse(ogNode.left, old_to_new)
    # CopyNode.colHeader = traverse(Node.colheader, old_to_new)
    # CopyNode.rowHeader = traverse(Node.rowheader, old_to_new)

    return CopyNode

def deep_copy_graph(Node):
    if not Node:
        return None

    old_to_new = {}
    return traverse(Node, old_to_new)

def guessing_game(targetNode, ColumnList, numbers, centralNode, Solutions):
    # create a copy of the solved matrix so far
    centralNodeCopy = deep_copy_graph(centralNode)
    reset(centralNode)
    ColumnListCopy = [None]*len(ColumnList)
    current = centralNodeCopy.down
    while (current != centralNodeCopy):
        ColumnListCopy[current.row] = current
        current = current.down

    SolutionsCopy = copy.deepcopy(Solutions)
    fix_row(ColumnListCopy,targetNode.row,SolutionsCopy)
    solve_as_much_as_possible(SolutionsCopy,centralNodeCopy,ColumnListCopy)
    
    if (len(SolutionsCopy) < numbers*numbers):
        target = minColumn(centralNodeCopy)
        guess = target.down
        for i in range(0,target.nodeCount):
            print(f"Entering the internal guessing game with {target.nodeCount} possibilities")
            guessing_game(guess,ColumnListCopy,numbers,centralNodeCopy,SolutionsCopy)
            if (len(SolutionsCopy) == numbers*numbers):
                break
            guess = guess.down
    if (len(SolutionsCopy) == numbers*numbers):
        print(SolutionsCopy)
        Solutions += SolutionsCopy[len(Solutions):]

def solve_sudoku(grid,ColumnList,numbers,centralNode,Solutions):
    rows = len(grid)
    columns = len(grid[0])
    for i in range(rows):
        for j in range(columns):
            value = grid[i][j]
            if value == 0:
                continue
            row_indice = numbers*numbers*i + numbers*j + value - 1
            fix_row(ColumnList,row_indice,Solutions)

    solve_as_much_as_possible(Solutions,centralNode,ColumnList)

    if (len(Solutions) == numbers*numbers):
        print(Solutions)
        return
    else:
        target = minColumn(centralNode)
        guess = target.down
        for i in range(0,target.nodeCount):
            print(f"Entering the guessing game with {target.nodeCount} possibilities")
            guessing_game(guess,ColumnList,numbers,centralNode,Solutions)
            if (len(Solutions) == numbers*numbers):
                break
            guess = guess.down

    print(Solutions)



In [100]:
numbers = 9
HansonMatrix = np.zeros((numbers*numbers*numbers,4*numbers*numbers))
testMatrix  = np.zeros((numbers*numbers*numbers,4*numbers*numbers))
grid = np.array([[1,0,0,4],[3,0,1,2],[4,0,2,0],[0,3,0,1]])
grid = np.array([[5,0,0,4,0,0,0,3,0],
                [0,0,0,0,1,0,6,0,0],
                [0,0,0,0,8,0,0,4,0],
                [0,0,0,0,0,0,0,0,0],
                [0,0,1,3,4,0,0,0,0],
                [0,7,3,2,0,0,0,0,9],
                [6,8,0,0,0,0,0,0,7],
                [0,0,0,5,0,0,0,2,0],
                [2,1,0,0,6,0,0,0,0]])
# grid = np.zeros((4,4)).astype(int)
# grid[0][0] = 1

columns = 4*numbers*numbers
rows = numbers*numbers*numbers

#initialize(HansonMatrix,numbers)

centralNode = Node(row=-1,column=-1,count=0)

HeaderList = []
HeaderList = [Node(row = -1, column= _ , count= 0) for _ in range(columns)]

ColumnList = []
ColumnList = [Node(row = _, column= -1, count= 0) for _ in range(rows)]


HeaderList[0].left = centralNode
centralNode.right = HeaderList[0]
HeaderList[0].right = HeaderList[1]
HeaderList[columns-1].right = centralNode
centralNode.left = HeaderList[columns -1]
HeaderList[columns-1].left = HeaderList[columns - 2]

for j in range(1,columns-1):
    HeaderList[j].left = HeaderList[j-1]
    HeaderList[j].right = HeaderList[j+1]

ColumnList[0].up = centralNode
centralNode.down = ColumnList[0]
ColumnList[0].down = ColumnList[1]
ColumnList[rows-1].down = centralNode
centralNode.up = ColumnList[rows -1]
ColumnList[rows-1].up = ColumnList[rows - 2]

for i in range(1,rows-1):
    ColumnList[i].up = ColumnList[i-1]
    ColumnList[i].down = ColumnList[i+1]

Solutions = []

initialize(HansonMatrix,numbers)
initial_links(HansonMatrix,HeaderList,ColumnList)
solve_sudoku(grid,ColumnList,numbers,centralNode,Solutions)


In [102]:
def convert_sol(Solutions,numbers):
    answer = np.zeros((numbers,numbers))
    for sol in Solutions:
        value = sol%numbers + 1
        column = math.floor(sol/numbers)%numbers 
        row = math.floor(sol/(numbers*numbers)) 
        #print(f"value at {row},{column} is {value}")
        answer[row][column] = value
    print(answer)
convert_sol(Solutions,numbers)

[[5. 6. 8. 4. 2. 7. 9. 3. 1.]
 [3. 4. 2. 9. 1. 5. 6. 7. 8.]
 [1. 9. 7. 6. 8. 3. 2. 4. 5.]
 [9. 2. 6. 8. 7. 1. 3. 5. 4.]
 [8. 5. 1. 3. 4. 9. 7. 6. 2.]
 [4. 7. 3. 2. 5. 6. 8. 1. 9.]
 [6. 8. 5. 1. 3. 2. 4. 9. 7.]
 [7. 3. 4. 5. 9. 8. 1. 2. 6.]
 [2. 1. 9. 7. 6. 4. 5. 8. 3.]]


In [ ]:
printHanson(testMatrix,centralNode)